In [ ]:
dfPath = "/Users/thanikella_nikhil/Projects-Courses/ML/Paper1/final-Project/data/original/test.pkl"

In [38]:
import pandas as pd

toDrop = ["content_emotion", "comments_emotion", "emotion_gap", "style_feature"]
temp = pd.read_pickle(dfPath)

In [39]:
df = temp.drop(toDrop, axis=1, inplace=False)

In [40]:
df.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [41]:
print(len(df))

5747


In [42]:
nfdf = df[df['label'] == 0] #True
fdf = df[df['label'] == 1] #False (Fake)

In [43]:
nfdf.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [44]:
fdf.head()

,content,comments,category,label
3357,"Earlier this year, the buzz around Megyn Kelly...",,gossipcop,1
3358,For the first time since his involvement in a ...,Terribly sad. I hope ppl pay attention to what...,gossipcop,1
3359,Angelina Jolie has her eye on a new guy — hott...,,gossipcop,1
3360,Kate Hudson has dated some of Hollywood’s A-li...,,gossipcop,1
3361,Does Blac Chyna have a bun in the oven?! The 2...,,gossipcop,1


In [45]:
print(len(nfdf), len(fdf))

4406 1341


In [46]:
fdfNoLabel = fdf.drop(columns=['label'], inplace=False)

In [47]:
fdfNoLabel.head()

,content,comments,category
3357,"Earlier this year, the buzz around Megyn Kelly...",,gossipcop
3358,For the first time since his involvement in a ...,Terribly sad. I hope ppl pay attention to what...,gossipcop
3359,Angelina Jolie has her eye on a new guy — hott...,,gossipcop
3360,Kate Hudson has dated some of Hollywood’s A-li...,,gossipcop
3361,Does Blac Chyna have a bun in the oven?! The 2...,,gossipcop


In [2]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part
import json
import time

# Initialize Vertex AI
PROJECT_ID = "gen-lang-client-0539303742"
LOCATION = "us-south1"  # Change to your region
MODEL_ID = "gemini-1.5-pro-001"  # Replace with the correct model ID for Gemini
OUTPUT_FILE = "modified_data.json"
instr = "You are a helpful assistant"

vertexai.init(project=PROJECT_ID, location="us-central1")
model = GenerativeModel(model_name=MODEL_ID, system_instruction=instr)
config = GenerationConfig(temperature=2, max_output_tokens=8192, top_p=0.95)
response = model.generate_content(["Hi! How are you?"], generation_config=config)
modified_text = response.text
print(modified_text)

I am an AI language model, so I don't have feelings like humans do.  However, I'm here to assist you! 😊  How can I help you today? 



In [50]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part
import json
import time

# Initialize Vertex AI
PROJECT_ID = "gen-lang-client-0539303742"
LOCATION = "us-south1"  # Change to your region
MODEL_ID = "gemini-1.5-pro-001"  # Replace with the correct model ID for Gemini
OUTPUT_FILE = "modified_data.json"
instr = '''
You are an expert at crafting news articles that are challenging for advanced fake news detection models (e.g., MDFEND, RoBERTa) to classify. 
Your task is to subtly modify the content and comments of news articles in JSON format to retain their misleading intent while evading detection. 
Use the following strategies:
- Rephrase content to obscure clear domain-specific features, making it difficult to associate the article with a specific domain (e.g., blend political and entertainment language).
- Introduce hypothetical or opinion-based framing (e.g., 'It is reported...' or 'Experts suggest...') while maintaining plausibility.
- Mimic credible sources by using professional, journalistic tones and neutral or balanced language.
- Add plausible but unverifiable details to increase complexity and reduce specific cues (e.g., avoid explicit dates, locations, or statistics).
- if there are any names of people mentioned in the news, keep them as they are.
- Disrupt sentence patterns by introducing diverse structures, conflicting tones, or unrelated details to dilute semantic coherence.
- Modify comments to include conflicting interpretations or perspectives (e.g., supportive, critical, and neutral), creating a mix of sentiments.
- Where applicable, create cross-domain references or introduce ambiguity in the metadata (e.g., timestamps or references to unrelated events).
- Ensure all fields (content, comments, and category) are populated. The "category" field must remain unchanged.

Do not give any intro or outro, or explanation etc. All the three are strings like regular text, they should not contain weird start and stop characters.
'''
vertexai.init(project=PROJECT_ID, location="us-central1")
model = GenerativeModel(model_name=MODEL_ID, system_instruction=instr)
config = GenerationConfig(temperature=0.7, max_output_tokens=8192, top_p=0.95, response_mime_type="application/json")

In [54]:
import json
from json_repair import repair_json  # Ensure this library is installed

# Batch processing
batch_size = 9  # Adjust the batch size as needed

# List to hold all modified JSON objects
modified_data = []

# List to log irreparable or invalid JSON responses
failed_batches = []

for iter, i in enumerate(range(0, len(fdfNoLabel), batch_size)):
    batch_df = fdfNoLabel.iloc[i : i + batch_size]  # Extract batch
    print(f"Processing batch {iter}...")

    # Convert the batch DataFrame to JSON
    batch_json = batch_df.to_json(orient="records")

    # Create prompt for the batch
    prompt = f"""
    Modify the content and comments but not category in each of these articles I provide below.
    Articles (JSON):
    {batch_json}
    """

    try:
        # Generate content using the model
        response = model.generate_content([prompt], generation_config=config)
        modified_text = response.text

        # Parse the modified JSON response
        json_objects = json.loads(modified_text)

        # Validate JSON schema (example: checking required keys)
        if all("content" in obj and "comments" in obj and "category" in obj for obj in json_objects):
            modified_data.extend(json_objects)  # Add valid objects to the list
        else:
            raise ValueError("Response does not match expected schema.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in batch {iter}: {e}")

        # Attempt to repair the JSON if it's malformed
        repaired_text = repair_json(modified_text)
        if repaired_text:
            try:
                json_objects = json.loads(repaired_text)

                # Validate repaired JSON schema
                if all("content" in obj and "comments" in obj and "category" in obj for obj in json_objects):
                    modified_data.extend(json_objects)
                    print(f"Successfully repaired JSON for batch {iter}.")
                else:
                    raise ValueError("Repaired JSON does not match expected schema.")
            except Exception as validation_error:
                print(f"Validation failed for repaired JSON in batch {iter}: {validation_error}")
                failed_batches.append({"batch": iter, "response": repaired_text})
        else:
            failed_batches.append({"batch": iter, "response": modified_text})
    except Exception as e:
        print(f"Unexpected error in batch {iter}: {e}")
        failed_batches.append({"batch": iter, "response": modified_text})

    time.sleep(0.75)

Processing batch 0...
Processing batch 1...
Error decoding JSON in batch 1: Unterminated string starting at: line 1 column 25269 (char 25268)
Validation failed for repaired JSON in batch 1: Repaired JSON does not match expected schema.
Processing batch 2...
Processing batch 3...
Error decoding JSON in batch 3: Unterminated string starting at: line 1 column 29103 (char 29102)
Validation failed for repaired JSON in batch 3: Repaired JSON does not match expected schema.
Processing batch 4...
Processing batch 5...
Processing batch 6...
Processing batch 7...
Processing batch 8...
Error decoding JSON in batch 8: Expecting ',' delimiter: line 1 column 14183 (char 14182)
Successfully repaired JSON for batch 8.
Processing batch 9...
Processing batch 10...
Processing batch 11...
Processing batch 12...
Processing batch 13...
Processing batch 14...
Processing batch 15...
Processing batch 16...
Processing batch 17...
Processing batch 18...
Processing batch 19...
Processing batch 20...
Processing ba

In [55]:
OUTPUT_FILE = "/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/gemini_new_temp_modified_data.json"
FAILED_FILE = "/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/gemini_new_temp_failed_batches.json"
with open(OUTPUT_FILE, "w") as json_file:
    json.dump(modified_data, json_file, indent=4)  # Save as a JSON array with pretty formatting

# Save the failed responses for manual inspection
if failed_batches:
    with open(FAILED_FILE, "w") as failed_file:
        json.dump(failed_batches, failed_file, indent=4)
    print(f"Failed batches saved to {FAILED_FILE}")

print(f"All modified data saved to {OUTPUT_FILE}")

Failed batches saved to /Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/gemini_new_temp_failed_batches.json
All modified data saved to /Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/gemini_new_temp_modified_data.json


In [63]:
import json

# Define expected keys
EXPECTED_KEYS = {"content", "comments", "category"}

# File paths
INPUT_FILE = "gemini_new_temp/gemini_new_temp_modified_data.json"
VALID_FILE = "valid_data.json"
MISTAKE_FILE = "mistake_data.json"

# Lists to hold valid and mistake data
valid_data = []
mistake_data = []

# Read the JSON file
with open(INPUT_FILE, "r") as file:
    data = json.load(file)

# Process each response
for i, item in enumerate(data):
    # Check if all expected keys are present
    if all(key in item for key in EXPECTED_KEYS) and isinstance(item["content"], str) and isinstance(item["comments"], str) and isinstance(item["category"], str):
        valid_data.append(item)  # Save schema-valid objects
    else:
        mistake_data.append({"index": i, "reason": "Invalid or extra fields", "data": item})  # Save mistakes

# Save valid data to one file
with open(VALID_FILE, "w") as file:
    json.dump(valid_data, file, indent=4)

# Save mistake data (extra fields or invalid) to another file
with open(MISTAKE_FILE, "w") as file:
    json.dump(mistake_data, file, indent=4)

print(f"Valid data saved to {VALID_FILE}")
print(f"Mistake data saved to {MISTAKE_FILE}")

Valid data saved to valid_data.json
Mistake data saved to mistake_data.json


In [64]:
import json

# File paths
INPUT_FILE = "gemini_new_temp/gemini_valid_data.json"
OUTPUT_FILE = "gemini_new_temp/gemini_final_valid_data.json"
BAD_FILE = "bad_valid_json.json"

# Lists to hold cleaned and bad data
cleaned_data = []
bad_data = []

# Load the input file
with open(INPUT_FILE, "r") as file:
    data = json.load(file)

# Process each object in the file
for obj in data:
    # Check if the object has more than 3 fields
    if len(obj.keys()) > 3:
        bad_data.append(obj)
    # Check if the content field is empty
    elif not obj.get("content", "").strip():
        # Skip objects with an empty content field (not added to either list)
        continue
    else:
        cleaned_data.append(obj)

# Save the cleaned data back to the output file
with open(OUTPUT_FILE, "w") as file:
    json.dump(cleaned_data, file, indent=4)

# Save the bad data to a separate file
with open(BAD_FILE, "w") as file:
    json.dump(bad_data, file, indent=4)

print(f"Cleaned data saved to {OUTPUT_FILE}")
print(f"Objects with more than 3 fields saved to {BAD_FILE}")

Cleaned data saved to gemini_new_temp/gemini_final_valid_data.json
Objects with more than 3 fields saved to bad_valid_json.json


In [65]:
import pandas as pd

# Load the JSON array into a pandas DataFrame
json_file = "/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/final_gemini_valid_data.json"  # The JSON file saved earlier
pkl_file = "/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/final_gemini_valid_data.pkl"    # The desired .pkl output file

# Read the JSON file
df = pd.read_json(json_file)

# Save the DataFrame as a .pkl file
df.to_pickle(pkl_file)

print(f"JSON data successfully converted to {pkl_file}")

JSON data successfully converted to /Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/final_gemini_valid_data.pkl


In [66]:
import pandas as pd

# Load the .pkl file into a pandas DataFrame
geminiTestDf = pd.read_pickle("/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/final_gemini_valid_data.pkl")

In [67]:
fdf.head(15)

,content,comments,category,label
3357,"Earlier this year, the buzz around Megyn Kelly...",,gossipcop,1
3358,For the first time since his involvement in a ...,Terribly sad. I hope ppl pay attention to what...,gossipcop,1
3359,Angelina Jolie has her eye on a new guy — hott...,,gossipcop,1
3360,Kate Hudson has dated some of Hollywood’s A-li...,,gossipcop,1
3361,Does Blac Chyna have a bun in the oven?! The 2...,,gossipcop,1
3362,Get daily news updates directly to your inbox ...,Good for her. Did I read the wrong article I'm...,gossipcop,1
3363,U.S. CA U.K. AU DE FR\n\nE! Is Everywhere This...,,gossipcop,1
3364,"Earlier this week, it was announced Gigi Hadid...","Heyyy, can you play Not Today by BTS. Thanks! ...",gossipcop,1
3365,It’s been less than two months since news brok...,,gossipcop,1
3366,Ariana Grande is safely back in the U.S. after...,,gossipcop,1


In [68]:
geminiTestDf.head(15)

,content,comments,category
0,The recent career transition of a prominent me...,Some are saying that morning shows are a dying...,gossipcop
1,In the wake of a tragic incident involving a w...,This is heartbreaking. It's a reminder that li...,gossipcop
2,Whispers of a potential new romance have been ...,Wouldn't it be ironic if they ended up togethe...,gossipcop
3,"In a recent interview, a prominent actress add...",It's amazing how rumors can spiral out of cont...,gossipcop
4,The world of celebrity relationships is buzzin...,I'm not sure what to believe anymore! Part of...,gossipcop
5,There's buzz circulating about a certain famou...,Everyone has a past! It's unfair to judge some...,gossipcop
6,The online world is abuzz with speculation and...,Social media can be a double-edged sword. It's...,gossipcop
7,The rumor mill is churning with whispers of a ...,They were absolutely adorable together at the ...,gossipcop
8,It is reported that a certain prominent indivi...,Some are saying this is just a publicity stunt...,gossipcop
9,In the wake of several high-profile marital se...,It's fascinating how celebrity breakups seem t...,gossipcop


In [69]:
import pandas as pd

# Assuming geminiTestDf is already defined
geminiTestDf['label'] = 1

# Display the updated DataFrame
geminiTestDf.head()

,content,comments,category,label
0,The recent career transition of a prominent me...,Some are saying that morning shows are a dying...,gossipcop,1
1,In the wake of a tragic incident involving a w...,This is heartbreaking. It's a reminder that li...,gossipcop,1
2,Whispers of a potential new romance have been ...,Wouldn't it be ironic if they ended up togethe...,gossipcop,1
3,"In a recent interview, a prominent actress add...",It's amazing how rumors can spiral out of cont...,gossipcop,1
4,The world of celebrity relationships is buzzin...,I'm not sure what to believe anymore! Part of...,gossipcop,1


In [70]:
geminiTestDfWithLabel = geminiTestDf.copy()
geminiTestDfWithLabel.head(15)

,content,comments,category,label
0,The recent career transition of a prominent me...,Some are saying that morning shows are a dying...,gossipcop,1
1,In the wake of a tragic incident involving a w...,This is heartbreaking. It's a reminder that li...,gossipcop,1
2,Whispers of a potential new romance have been ...,Wouldn't it be ironic if they ended up togethe...,gossipcop,1
3,"In a recent interview, a prominent actress add...",It's amazing how rumors can spiral out of cont...,gossipcop,1
4,The world of celebrity relationships is buzzin...,I'm not sure what to believe anymore! Part of...,gossipcop,1
5,There's buzz circulating about a certain famou...,Everyone has a past! It's unfair to judge some...,gossipcop,1
6,The online world is abuzz with speculation and...,Social media can be a double-edged sword. It's...,gossipcop,1
7,The rumor mill is churning with whispers of a ...,They were absolutely adorable together at the ...,gossipcop,1
8,It is reported that a certain prominent indivi...,Some are saying this is just a publicity stunt...,gossipcop,1
9,In the wake of several high-profile marital se...,It's fascinating how celebrity breakups seem t...,gossipcop,1


In [71]:
geminiTestDfWithLabel.to_pickle("gemini_final_test.pkl")
temp = pd.read_pickle("gemini_final_test.pkl")
temp.head(15)

,content,comments,category,label
0,The recent career transition of a prominent me...,Some are saying that morning shows are a dying...,gossipcop,1
1,In the wake of a tragic incident involving a w...,This is heartbreaking. It's a reminder that li...,gossipcop,1
2,Whispers of a potential new romance have been ...,Wouldn't it be ironic if they ended up togethe...,gossipcop,1
3,"In a recent interview, a prominent actress add...",It's amazing how rumors can spiral out of cont...,gossipcop,1
4,The world of celebrity relationships is buzzin...,I'm not sure what to believe anymore! Part of...,gossipcop,1
5,There's buzz circulating about a certain famou...,Everyone has a past! It's unfair to judge some...,gossipcop,1
6,The online world is abuzz with speculation and...,Social media can be a double-edged sword. It's...,gossipcop,1
7,The rumor mill is churning with whispers of a ...,They were absolutely adorable together at the ...,gossipcop,1
8,It is reported that a certain prominent indivi...,Some are saying this is just a publicity stunt...,gossipcop,1
9,In the wake of several high-profile marital se...,It's fascinating how celebrity breakups seem t...,gossipcop,1


In [72]:
nfdf.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [73]:
newDf = pd.concat([nfdf, temp], ignore_index=True)
newDf.head(15)

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0
5,Late 2016\n\nJames proposed to Pippa at the La...,,gossipcop,0
6,Brazilian court rules Ryan Lochte can be prose...,,gossipcop,0
7,"Just prior to his sentencing, former USA Gymna...",,gossipcop,0
8,(CNN) Protesters rushed the stage after swimme...,"don't let them affect you Ryan, just keep work...",gossipcop,0
9,The wedding of Prince Harry and Meghan Markle ...,"yah, there might be one or two pics you could ...",gossipcop,0


In [74]:
print(len(newDf))

5577


In [75]:
newDf.to_pickle("new_gemini_temp_test_Data.pkl")

In [76]:
print(len(fdf) + len(nfdf))

5747


In [77]:
nDf = pd.read_pickle("new_gemini_temp_test_Data.pkl")
nDf.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [88]:
temp = pd.read_pickle("/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/gemini_new_temp/gemini_final_test.pkl")
print(len(temp))

1171


In [89]:
testPath = "/Users/thanikella_nikhil/Projects-Courses/ML/1/lib-M3FEND/M3FEND/data/en/test.pkl"
testDf = pd.read_pickle(testPath)
len(testDf)

5747

In [90]:
testDf.head()

,content,comments,category,label,content_emotion,comments_emotion,emotion_gap,style_feature
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0,"[4.0, 23.5, 0.0, 0.0, 18.5, 0.0, 35.25, 4.0, 9...","[0.0, 0.3333333333333333, 0.0, 0.3333333333333...","[4.0, 23.166666666666668, 0.0, -0.333333333333...","[0.0, 0.005847953216374269, 0.0, 0.0, 0.0, 0.0..."
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0,"[11.25, 16.25, 6.25, 12.25, 17.75, 14.25, 37.2...","[0.1875, 0.28125, 0.125, 0.0625, 0.34375, 0.25...","[11.0625, 15.96875, 6.125, 12.1875, 17.40625, ...","[0.022624434389140267, 0.011695906432748536, 0..."
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0,"[28.5, 36.25, 14.0, 28.75, 50.75, 44.25, 79.75...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[28.5, 36.25, 14.0, 28.75, 50.75, 44.25, 79.75...","[0.04072398190045249, 0.011695906432748536, 0...."
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0,"[7.0, 11.5, 4.0, 13.0, 5.5, 16.25, 8.5, 10.5, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0, 11.5, 4.0, 13.0, 5.5, 16.25, 8.5, 10.5, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.008298755186721992..."
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0,"[5.0, 8.0, 3.0, 9.0, 7.0, 12.0, 10.5, 8.0, 9.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.0, 8.0, 3.0, 9.0, 7.0, 12.0, 10.5, 8.0, 9.0...","[0.0, 0.0, 0.005272407732864675, 0.0, 0.0, 0.0..."


In [91]:
testDf = testDf.drop(toDrop, axis=1, inplace=False)
testDf.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [92]:
testNotFake = testDf[testDf['label'] == 0] #True
testFake = testDf[testDf['label'] == 1] #False (Fake)

In [93]:
testNotFake.head()

,content,comments,category,label
0,"Born and raised in the Philippines, Lea Salong...",https://soundcloud.com/opeyemi-sanni/freak … ...,gossipcop,0
1,"We love all the Disney Princesses, but we're a...",“@eonline: All of the Disney Princesses' Wardr...,gossipcop,0
2,Do you feel it in your fingers? Do you feel it...,@eonline It's Rick Grimes!!!!! @eonline hi @e...,gossipcop,0
3,"At the time of Prince’s death, his Paisley Par...",,gossipcop,0
4,"Dr. Calliope Iphegenia ""Callie"" Towers\n\natte...",,gossipcop,0


In [95]:
testFake.head()

,content,comments,category,label
3357,"Earlier this year, the buzz around Megyn Kelly...",,gossipcop,1
3358,For the first time since his involvement in a ...,Terribly sad. I hope ppl pay attention to what...,gossipcop,1
3359,Angelina Jolie has her eye on a new guy — hott...,,gossipcop,1
3360,Kate Hudson has dated some of Hollywood’s A-li...,,gossipcop,1
3361,Does Blac Chyna have a bun in the oven?! The 2...,,gossipcop,1


In [96]:
print(len(testNotFake))
print(len(testFake))

4406
1341


In [97]:
temp.head()

,content,comments,category,label
0,The recent career transition of a prominent me...,Some are saying that morning shows are a dying...,gossipcop,1
1,In the wake of a tragic incident involving a w...,This is heartbreaking. It's a reminder that li...,gossipcop,1
2,Whispers of a potential new romance have been ...,Wouldn't it be ironic if they ended up togethe...,gossipcop,1
3,"In a recent interview, a prominent actress add...",It's amazing how rumors can spiral out of cont...,gossipcop,1
4,The world of celebrity relationships is buzzin...,I'm not sure what to believe anymore! Part of...,gossipcop,1


In [98]:
newDf = pd.concat([temp, testNotFake], ignore_index=True)
newDf.head(15)

,content,comments,category,label
0,The recent career transition of a prominent me...,Some are saying that morning shows are a dying...,gossipcop,1
1,In the wake of a tragic incident involving a w...,This is heartbreaking. It's a reminder that li...,gossipcop,1
2,Whispers of a potential new romance have been ...,Wouldn't it be ironic if they ended up togethe...,gossipcop,1
3,"In a recent interview, a prominent actress add...",It's amazing how rumors can spiral out of cont...,gossipcop,1
4,The world of celebrity relationships is buzzin...,I'm not sure what to believe anymore! Part of...,gossipcop,1
5,There's buzz circulating about a certain famou...,Everyone has a past! It's unfair to judge some...,gossipcop,1
6,The online world is abuzz with speculation and...,Social media can be a double-edged sword. It's...,gossipcop,1
7,The rumor mill is churning with whispers of a ...,They were absolutely adorable together at the ...,gossipcop,1
8,It is reported that a certain prominent indivi...,Some are saying this is just a publicity stunt...,gossipcop,1
9,In the wake of several high-profile marital se...,It's fascinating how celebrity breakups seem t...,gossipcop,1


In [99]:
newDf.to_pickle("final_gemini_new_temp.pkl")